# Uma análise do mercado Segurador brasileiro

## Perguntas referencia
1. Crescimento entre seguradoras por premio ganho e emitido
2. top premio emitido
3. Market Share regra do link
4. Indice de Eficiencia : Despesa Administrativa/premio ganho
5. Indice de Eficiencia : Despesa Administrativa/premio emitido
6. indice de sinistralidade: sinistros ocorridos / premio ganho
7. premio ganho + premio ganho risco / custo de aquisicao risco + custo de aquisicao normal
8. combined: sinistralidade + custo de aquisicao / premio ganho

## 1. Set up

Import some packages and the files I am goint to use.

In [73]:
#import the libraries
import pandas as pd
import pyodbc
import os.path
import numpy as np
import datetime
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', None)

In [74]:
path = '/Users/Lucas/Documents/SES-SUSEP/'

#income statments sheet
income_statment = pd.read_csv(path+'SES_Balanco.csv', ';',decimal =',',dtype={'valor': np.float64})

#companies
companies = pd.read_csv(path+'SES_cias.csv', ';',encoding='latin-1')

#encode for the income statments base fields 
fields = pd.read_csv(path+'SES_campos.csv', ';',encoding='latin-1')

#values of prize by branch
premium = pd.read_csv(path+'Ses_seguros.csv', ';',encoding='latin-1')

#comanies holdings
company_goup = pd.read_csv(path+'Ses_grupos_economicos.csv', ';',encoding='latin-1')


/Users/Lucas/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2. Exploratory analysis

I am going to do a Exploratory analysis over the bases I imported. Important statments:
 - The fields base has a end of term column. The last end of term is 210001 which stand for year 2020 and month 01 (janary)
 - the income_statmente has only one field tha should be of type float, which is 'values'
 - the income_statment is showing the cumulative sum of the values of each field
 - the are some Nan values in the premium dataset. 


In [75]:
#income_statment.info()
#companies.head()
#fields.fimvigencia.unique()
#fields.info()
#max(set(fields.fimvigencia)) ## 201906
fields[fields.noitem.str.contains(regex=True,pat='(Donativos e Contribuições)')].nuitem.unique()
company_goup.head()

/Users/Lucas/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,damesano,coenti,noenti,cogrupo,nogrupo
0,200405,1007,SABEMI SEGURADORA SA ...,1225,INDEPENDENTE
1,200708,1619,DAYPREV VIDA E PREVIDÊNCIA S.A. ...,99999,OUTROS GRUPOS
2,201001,2062,LUIZASEG SEGUROS S/A ...,1203,CARDIF
3,200602,2143,ASSURANT SEGURADORA SA ...,1225,INDEPENDENTE
4,200709,2895,Alfa Previdência e Vida S.A. ...,1192,ALFA


## 3. Clean the base

The premium base is the core of our dataset and holds several importate rubrics for our analysis separated by branch. It has the following columns of interest:
    - damesano: year-month 202005
    - coenti: companies code
    - cogrupo: insurancer holding
    - coramo: branch's code :first 2 digits are the branch group and the last 2 are the specific branch
    - premio_emitido2: the premium 
    - premio_ganho: Earned Premium, according to the National Insurance brokers association (NIBA) glossary:
        
        "Insurance policies usually run for a period of 12 months. An insured
         can cancel a policy at any time and request a refund of premium.
         Therefore, insurers must only take into the books of account that
         portion of premium which corresponds to actual elapsed time on risk.
         That portion of premium which can be taken up in the accounts is
         called earned premium. That portion of premium yet to expire is"
        
    - sinistro_ocorrido: insurances claimed
    - desp_com: comercial expanses, like comission

I'll start separating 1 column for all the branches that a company work and another column for just the life branches. 

For life branch's we need to filter the first two coramo digits to 9,13 and only the full 'coramo' code 1198. The code 1198 is a specific branch called "farmer's life insurance". By some reason, not in the regular branches 9 and 13.

In all the dataset's, when possible, I selected the dates after december 2016 (201612).


In [76]:
premium['damesano'] = pd.to_datetime(premium['damesano'],format='%Y%m')
premium['year'] = premium.damesano.dt.year 
premium = premium[premium.year>2016]

#there are few nan values in the dataset. There are no big disadvantages in replacing it for 0's
premium.fillna(0,inplace=True)

life_premium = premium[(premium.coramo.astype(int).astype(str).str[:-2].astype(int).isin([9,13])) 
                  | (premium.coramo==1198) ]

premium_comparison = life_premium.merge(premium[['coenti','damesano','premio_emitido2']]
                                        ,left_on=['coenti','damesano']
                                        ,right_on=['coenti','damesano']
                                        ,suffixes=['_life','_all'])

keep_columns = ['coenti','year','premio_emitido2_life','premio_emitido2_all','premio_ganho','sinistro_ocorrido','desp_com']
premium_comparison = premium_comparison.filter(keep_columns,axis=1)

for var in premium_comparison.columns.tolist()[2:]:
    premium_comparison[var] = pd.to_numeric(premium_comparison[var].str.replace(',','.'))

#change names for easy understanding
premium_comparison.rename({
    "premio_emitido2_life":"premium_life",
    "premio_emitido2_all": "premium_all",
    "premio_ganho": "earned_life_premium",
    "sinistro_ocorrido": "insurance_claimed",
    "desp_com":"comercial_expanses"},
    inplace=True)

#aggregate the values for year and company
premium_comparison = premium_comparison.groupby(['coenti','year']).sum().reset_index() 

premium_comparison.head()

,coenti,year,premio_emitido2_life,premio_emitido2_all,premio_ganho,sinistro_ocorrido,desp_com
0,1007,2017,6492880430.52,2422861108.58,6480397898.37,-4011254.28,1066237306.53
1,1007,2018,7430172731.85,2690184261.93,7449256688.61,69660783.72,1645361612.40
2,1007,2019,5124562918.20,1808445012.66,5122519567.11,199328991.84,1467932244.24
3,1007,2020,1531974128.85,510658042.95,1534196795.67,70370556.63,379107153.81
4,1015,2020,92326459.05,25848055155.00,93251462.85,57702123.15,25697163.45


The income_statmente dataset contains:
    - coenti: companies code
    - damesano: year-month 2020-05
    - cmpid: field code in the income_statment
    - valor: values
    - seq: order of the field in the income_statment
    - quadro: type of accounting statement
    

For the income_statment base I filter the column 'quadro' to bring onlt the ones with ' 23' because this the code for the sheet of the income statment. Also, I filtered only the rubrics we need for the analyses:
 - 4069: that is the administrative expense (ae)
 - 6314: employee expenses 
 - 6315: third-party spending
 - 6317: publicity
 - 6319: donations
 - 7220: premium for risk type of insurence 

In [77]:
income_statment = income_statment[(income_statment['damesano']>201612) &
                      (income_statment['quadro'] == '23 ') &
                      (income_statment.cmpid.isin([4069,6314,6315,7220]))
                     ] 

# I prefered to work with the income_statment in wide format rather then long, so I converted the data frame.
income_statment = pd.pivot_table(income_statment, index=['coenti','damesano'],columns='cmpid', values='valor').rename_axis(None, axis=1).reset_index()
income_statment.rename(columns={4069: "ae",
                6314: "ae_employee",
                6315: "ae_3pspending",
                6317: "ae_publicity",
                6319: "ae_donations",
                7220: "premium_risk"}
       ,inplace=True )

#filtering the last period of each year
income_statment ['damesano'] = pd.to_datetime(income_statment['damesano'],format='%Y%m')
income_statment['year'] = income_statment.damesano.dt.year
last_is = income_statment.groupby(['coenti','year']).max().reset_index()[['coenti','damesano']]
income_statment = income_statment.merge(last_is,left_on=['coenti','damesano'],right_on=['coenti','damesano'])

income_statment.drop('damesano',axis=1,inplace=True)
income_statment.head()


,coenti,ae,ae_employee,ae_3pspending,premium_risk,year
0,1007,-84945871.94,-32670914.56,-27031670.20,15692215.15,2017
1,1007,-110979218.53,-41663000.21,-34792803.26,21743971.25,2018
2,1007,-120870713.01,-40558909.19,-38199930.77,24594166.06,2019
3,1007,-39285988.54,-12235318.12,-14653105.72,11252190.77,2020
4,1015,-99919567.14,-66467556.99,-19859984.50,0.00,2020


The company_group dataset contains:
    - coenti
    - damesano: year-month of that group info to the company
    - noenti: name of company
    - cogrupo: code of company group
    - nogrupo: name of company group

In [78]:
#chosing only the latest group names
company_goup_key = company_goup.groupby('coenti').max().reset_index()[['coenti','damesano']]
company_goup = company_goup.merge(company_goup_key,how='inner',left_on=['coenti','damesano'],right_on=['coenti','damesano'])

#if the group is independent
company_goup.nogrupo = np.where(company_goup.cogrupo.isin(['1225','99999']),
                                company_goup.noenti.str.strip(),
                                company_goup.nogrupo.str.strip())
company_goup = company_goup[['coenti','nogrupo']]

#this comapnies belong to the same group but they weren't grouped. I'll do it manualy 
company_goup.replace(['Zurich Santander Brasil Seguros e Previdencia S/A','ZURICH BRASIL COMPANHIA DE SEGUROS'],
                     'ZURICH',
                     inplace=True)


In [82]:
final_dataset = premium_comparison.merge(income_statment,left_on=['coenti','year'],right_on=['coenti','year'])
final_dataset = final_dataset.merge(company_goup,left_on='coenti',right_on='coenti')

ae_proportion = final_dataset.premio_emitido2_life/final_dataset.premio_emitido2_all
final_dataset['OEI'] = final_dataset.ae * ae_proportion / final_dataset.premio_emitido2_life
final_dataset.head()

,coenti,year,premio_emitido2_life,premio_emitido2_all,premio_ganho,sinistro_ocorrido,desp_com,ae,ae_employee,ae_3pspending,premium_risk,nogrupo,OEI
0,1007,2017,6492880430.52,2422861108.58,6480397898.37,-4011254.28,1066237306.53,-84945871.94,-32670914.56,-27031670.20,15692215.15,SABEMI SEGURADORA SA,-0.04
1,1007,2018,7430172731.85,2690184261.93,7449256688.61,69660783.72,1645361612.40,-110979218.53,-41663000.21,-34792803.26,21743971.25,SABEMI SEGURADORA SA,-0.04
2,1007,2019,5124562918.20,1808445012.66,5122519567.11,199328991.84,1467932244.24,-120870713.01,-40558909.19,-38199930.77,24594166.06,SABEMI SEGURADORA SA,-0.07
3,1007,2020,1531974128.85,510658042.95,1534196795.67,70370556.63,379107153.81,-39285988.54,-12235318.12,-14653105.72,11252190.77,SABEMI SEGURADORA SA,-0.08
4,1015,2020,92326459.05,25848055155.00,93251462.85,57702123.15,25697163.45,-99919567.14,-66467556.99,-19859984.50,0.00,SUL AMERICA,-0.00


In [510]:

#criando indicadores
comparativo_ramo.premio_ganho = comparativo_ramo.premio_ganho + comparativo_ramo.premio_risco
comparativo_ramo.premio_ganho_todos_ramos = comparativo_ramo.premio_risco + comparativo_ramo.premio_ganho_todos_ramos
proporcao_receita = (comparativo_ramo.premio_ganho + comparativo_ramo.premio_vgbl) / (comparativo_ramo.premio_ganho_todos_ramos + comparativo_ramo.premio_vgbl)
premio_ganho_total = comparativo_ramo.premio_ganho + comparativo_ramo.premio_vgbl
comparativo_ramo['da_proporcional'] = comparativo_ramo.da * proporcao_receita
comparativo_ramo['IEO'] = (-comparativo_ramo.da_proporcional / premio_ganho_total) * 100
comparativo_ramo['IEO_inteira'] = (-comparativo_ramo.da / premio_ganho_total) * 100
comparativo_ramo['CA/Pe'] = (comparativo_ramo.desp_com / comparativo_ramo.premio_direto) *  100
comparativo_ramo['combined'] = ((comparativo_ramo.desp_com + comparativo_ramo.sinistro_ocorrido) / comparativo_ramo.premio_ganho )*  100
comparativo_ramo['Sinistralidade'] = (comparativo_ramo.sinistro_ocorrido / comparativo_ramo.premio_ganho) * 100


comparativo_ramo = comparativo_ramo[comparativo_ramo.premio_ganho>0]
comparativo_ramo.drop(['cogrupo','coramo','Coenti','Nogrupo'],axis=1,inplace=True)
#'desp_com', 'sinistro_ocorrido', 'CA/Pe', 'combined', 'Sinistralidade', 'premio_direto'
comparativo_ramo = comparativo_ramo[['coenti','nogrupo', 'Noenti', 'damesano', 'premio_ganho','premio_risco','da', 'premio_vgbl', 'premio_ganho_todos_ramos', 'da_proporcional', 'IEO', 'IEO_inteira']]
comparativo_ramo.to_excel('G:/04 - Gerência de Produtos e Processos de Seguros/4.3 Informações Gerenciais/6. Mercado Segurador/SES-Susep/comparativo_ramo.xls',float_format='%.2f',inf_rep='',merge_cells=False,index=False)



C:\Users\lucas.correa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\lucas.correa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lucas.correa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [538]:
SES_campos.head()
#[6314, 6591] pessoal próprio
#[6315, 6592] serviços de terceiros
DRE_1[DRE_1.nuitem.isin([6315])]
DRE[['noitem','nuitem','seq']].drop_duplicates().sort_values('seq')
SES_campos[SES_campos.nuitem.isin([6314, 6591])]
SES_campos[SES_campos.noitem.str.contains(regex=True,pat='(Donativos e Contribuições)')].nuitem.unique()

array([6319, 6596], dtype=int64)